In [30]:
import os
import shapely
import numpy as np
import sys 
import pandas as pd 
import gc
from shapely import wkt
from matplotlib import pyplot as plt
from shapely.geometry import Point,mapping,Polygon,box,MultiPoint
import geopandas as gpd 
import seaborn as sns 

print(pd.__version__)
 
# for parallelization
sys.path.append('/mnt/c/Users/tandon/OneDrive - TomTom/Desktop/tomtom/Workspace/01_Rooftop_accuracy/APT-Realignment/src/apt_realignment')
from process_geometries import ProcessGeometricData  as ProcessAPT_data
from utils.haversine_distance import get_distance
from utils.geometric_utils import geojson2shpfile


def save_geometry(dataframe,geometry,out_path,fname='dataframe.geojson'):
    df_to_save = gpd.GeoDataFrame(geometry=dataframe[geometry],crs="EPSG:4326")
    df_to_save = df_to_save.to_crs("epsg:4326")
    df_to_save.to_file(os.path.join(out_path,fname),driver="GeoJSON")

1.2.5


In [31]:
from glob import glob 

data_path = "/mnt/c/Users/tandon/OneDrive - TomTom/Desktop/tomtom/Workspace/01_Rooftop_accuracy/APT-Realignment/data"
country = "Texas"
# read pkl data
pkl_path = os.path.join(data_path,country,'Apt_realignment_MSFT/')
pkl_files = glob(pkl_path+'*/*.pkl')

print("NUmber of pkl files found ",len(pkl_files))

NUmber of pkl files found  451


### Merge data

In [ ]:
def read_data(fname):
    df_bfp = pd.read_pickle(fname)
    df_bfp.reset_index(drop=True)
    return df_bfp

for fname in pkl_files:
    try:
        out = read_data(fname)
    except:
        print(fname)
# out = pd.concat([read_data(fname) for fname in pkl_files])

In [32]:
read_data("/mnt/c/Users/tandon/OneDrive - TomTom/Desktop/tomtom/Workspace/01_Rooftop_accuracy/APT-Realignment/data/Texas/Apt_realignment_MSFT/updated_geometries_bfp-count_1_Parcels_48279/APT_realigned.pkl")

AttributeError: Can't get attribute '_unpickle_block' on <module 'pandas._libs.internals' from '/home/tandon/anaconda3/envs/pygeo/lib/python3.8/site-packages/pandas/_libs/internals.cpython-38-x86_64-linux-gnu.so'>

In [ ]:
## Show single bfp within Parcel data
df_one_bfp = pd.read_pickle(pkl_files[0])
df_one_bfp = df_one_bfp.reset_index(drop=True)
print("simple scenario sahpe",df_one_bfp.shape[0])

df_two_bfp = pd.read_pickle(pkl_files[1])
df_two_bfp = df_two_bfp.reset_index(drop=True)
print(" complx scenario shape:",df_two_bfp.shape[0])

merge_df= pd.concat([df_one_bfp,df_two_bfp])
print("Merged Shape",merge_df.shape)
merge_df.to_pickle(os.path.join(pkl_path,'FinalUpdated_APT_{}_{}.pkl'.format(state,city)))

## IIlunois Cook

In [ ]:
import os
import shapely
import numpy as np
import sys 
import pycoredb 
import pandas as pd 
import gc
from shapely import wkt
from matplotlib import pyplot as plt
from shapely.geometry import Point,mapping,Polygon,box,MultiPoint
import geopandas as gpd 
import seaborn as sns 
 
# for parallelization
sys.path.append('/mnt/c/Users/tandon/OneDrive - TomTom/Desktop/tomtom/Workspace/01_Rooftop_accuracy/APT-Realignment/src/apt_realignment')
from process_geometries import ProcessGeometricData  as ProcessAPT_data
from utils.haversine_distance import get_distance
from utils.geometric_utils import geojson2shpfile

data_path  = "/mnt/c/Users/tandon/OneDrive - TomTom/Desktop/tomtom/Workspace/01_Rooftop_accuracy/BFP_Analysis_USA/data/data"
state = "Illinois"
city = "cook"
apt_data_path = os.path.join(data_path,state,"APT__2022_09_001_nam_usa_uil.shp" )
parcel_path = os.path.join(data_path,state,city,"Parcels_17031/Parcels_17031.shp")
building_geojson = os.path.join(data_path,state,'Illinois.geojson')

In [ ]:
# geojson2shpfile(building_geojson,verbose=0)

### Single BFP within a Parcel 

In [ ]:
output_path = os.path.join(data_path,state,city,'Apt_realignment') 
apt_preprocess = ProcessAPT_data(parcel_shapefile=parcel_path,
                building_shapefile=building_geojson,
                apt_shape_file=apt_data_path , 
                output_path=output_path)

processed_df_cook = apt_preprocess.process_dataframe(bfp_count_per_parcel=1,filename='APT_realigned_usa_il_cook')


In [ ]:
print(processed_df_cook.shape[0])
processed_df_cook.head(4)

In [ ]:
distance_greater_than_50m = processed_df_cook.loc[processed_df_cook['APT_to_Centroid_distance']>50].shape[0]
print("Data points greater than 50m: {:.2f}%".format(100*(distance_greater_than_50m/processed_df_cook.shape[0])))

distance_greater_than_50m = processed_df_cook.loc[processed_df_cook['APT_to_Centroid_distance']<50].shape[0]
print("Data points less than 50m: {:.2f}%".format(100*(distance_greater_than_50m/processed_df_cook.shape[0])))


In [ ]:
fig, ax = plt.subplots(figsize=(17,9))

ax.set_title("Histogram of APT to centroid distance on/not on BFP")
ax.set_xlabel("APT point to centroid distance(meters)")
ax.set_ylabel("counts")

frqTrue, edgesTrue = np.histogram(processed_df_cook['APT_to_Centroid_distance'].loc[processed_df_cook['APT_to_Centroid_distance']<150].values, bins = np.arange(0,150,10))
p1 = ax.bar(edgesTrue[:-1], frqTrue, width=np.diff(edgesTrue), edgecolor="black", align="edge",alpha=0.4,label='Address Points on Rooftop',color='orange')

plt.legend()
plt.show()
gc.collect()

#### Two BFP's within a Parcel 

In [ ]:
## Process complexity 2-- 2 bfp 

output_path = os.path.join(data_path,state,city,'Apt_realignment') 
apt_preprocess = ProcessAPT_data(parcel_shapefile=parcel_path,
                building_shapefile=building_geojson,
                apt_shape_file=apt_data_path , 
                output_path=output_path)

processed_df_cook = apt_preprocess.process_dataframe(bfp_count_per_parcel=2,filename='APT_realigned_usa_uil_cook')

In [ ]:
print(processed_df_cook.shape[0])
processed_df_cook.head(4)

In [ ]:
distance_greater_than_50m = processed_df_cook.loc[processed_df_cook['APT_to_Centroid_distance']>50].shape[0]
print("Data points greater than 50m: {:.2f}%".format(100*(distance_greater_than_50m/processed_df_cook.shape[0])))

distance_greater_than_50m = processed_df_cook.loc[processed_df_cook['APT_to_Centroid_distance']<50].shape[0]
print("Data points less than 50m: {:.2f}%".format(100*(distance_greater_than_50m/processed_df_cook.shape[0])))


In [ ]:
fig, ax = plt.subplots(figsize=(17,9))

ax.set_title("Histogram of APT to centroid distance on/not on BFP")
ax.set_xlabel("APT point to centroid distance(meters)")
ax.set_ylabel("counts")

frqTrue, edgesTrue = np.histogram(processed_df_cook['APT_to_Centroid_distance'].loc[processed_df_cook['APT_to_Centroid_distance']<150].values, bins = np.arange(0,150,10))
p1 = ax.bar(edgesTrue[:-1], frqTrue, width=np.diff(edgesTrue), edgecolor="black", align="edge",alpha=0.4,label='Address Points on Rooftop',color='orange')

plt.legend()
plt.show()
gc.collect()

### Merge Data

In [ ]:
from glob import glob 
pkl_path = osp.join(data_path,state,city,'Apt_realignment')
pkl_files = glob(pkl_path+'/*/*.pkl')
print(pkl_files)


In [ ]:
## Show single bfp within Parcel data
df_one_bfp = pd.read_pickle(pkl_files[0])
df_one_bfp = df_one_bfp.reset_index(drop=True)
print("simple scenario sahpe",df_one_bfp.shape[0])

df_two_bfp = pd.read_pickle(pkl_files[1])
df_two_bfp = df_two_bfp.reset_index(drop=True)
print(" complx scenario shape:",df_two_bfp.shape[0])

merge_df= pd.concat([df_one_bfp,df_two_bfp])
print("Merged Shape",merge_df.shape)
merge_df.to_pickle(os.path.join(pkl_path,'FinalUpdated_APT_{}_{}.pkl'.format(state,city)))


## Michigan Wayne 

In [ ]:
import os
import shapely
import numpy as np
import sys 
import pycoredb 
import pandas as pd 
import gc
from shapely import wkt
from matplotlib import pyplot as plt
from shapely.geometry import Point,mapping,Polygon,box,MultiPoint
import geopandas as gpd 
import seaborn as sns 
 
# for parallelization
sys.path.append('/mnt/c/Users/tandon/OneDrive - TomTom/Desktop/tomtom/Workspace/01_Rooftop_accuracy/APT-Realignment/src/apt_realignment')
from process_geometries import ProcessGeometricData  as ProcessAPT_data
from utils.haversine_distance import get_distance

data_path  = "/mnt/c/Users/tandon/OneDrive - TomTom/Desktop/tomtom/Workspace/01_Rooftop_accuracy/BFP_Analysis_USA/data/data"
state = "MI"
city = "wayne"
apt_data_path = os.path.join(data_path,state,"APT__2022_09_001_nam_usa_umx.shp" )
parcel_path = os.path.join(data_path,state,city,"Parcels_26163/Parcels_26163.shp")
building_geojson = os.path.join(data_path,state,'Michigan.geojson')

In [ ]:
# geojson2shpfile(building_geojson,verbose=0)

### Single BFP within a Parcel

In [ ]:
output_path = os.path.join(data_path,state,city,'Apt_realignment') 
apt_preprocess = ProcessAPT_data(parcel_shapefile=parcel_path,
                building_shapefile=building_geojson,
                apt_shape_file=apt_data_path , 
                output_path=output_path)

processed_df_wayne = apt_preprocess.process_dataframe(bfp_count_per_parcel=1,filename='APT_realigned_usa_umx_wayne')

In [ ]:
print("Data Shape:",processed_df_wayne.shape)
processed_df_wayne.head()

In [ ]:
distance_greater_than_50m = processed_df_wayne.loc[processed_df_wayne['APT_to_Centroid_distance']>50].shape[0]
print("Data points greater than 50m: {:.2f}%".format(100*(distance_greater_than_50m/processed_df_wayne.shape[0])))

distance_greater_than_50m = processed_df_wayne.loc[processed_df_wayne['APT_to_Centroid_distance']<50].shape[0]
print("Data points less than 50m: {:.2f}%".format(100*(distance_greater_than_50m/processed_df_wayne.shape[0])))

In [ ]:
fig, ax = plt.subplots(figsize=(17,9))

ax.set_title("Histogram of APT to centroid distance on/not on BFP")
ax.set_xlabel("APT point to centroid distance(meters)")
ax.set_ylabel("counts")

frqTrue, edgesTrue = np.histogram(processed_df_wayne['APT_to_Centroid_distance'].loc[processed_df_wayne['APT_to_Centroid_distance']<150].values, bins = np.arange(0,150,10))
p1 = ax.bar(edgesTrue[:-1], frqTrue, width=np.diff(edgesTrue), edgecolor="black", align="edge",alpha=0.4,label='Address Points on Rooftop',color='orange')

plt.legend()
plt.show()
gc.collect()

#### Two BFP's within a Parcel 

In [ ]:
output_path = os.path.join(data_path,state,city,'Apt_realignment') 
apt_preprocess = ProcessAPT_data(parcel_shapefile=parcel_path,
                building_shapefile=building_geojson,
                apt_shape_file=apt_data_path , 
                output_path=output_path)

processed_df_wayne = apt_preprocess.process_dataframe(bfp_count_per_parcel=2,filename='APT_realigned_usa_umx_wayne')


In [ ]:
print(processed_df_wayne.shape[0])
processed_df_wayne.head(10)

In [ ]:
distance_greater_than_50m = processed_df_wayne.loc[processed_df_wayne['APT_to_Centroid_distance']>50].shape[0]
print("Data points greater than 50m: {:.2f}%".format(100*(distance_greater_than_50m/processed_df_wayne.shape[0])))

distance_greater_than_50m = processed_df_wayne.loc[processed_df_wayne['APT_to_Centroid_distance']<50].shape[0]
print("Data points less than 50m: {:.2f}%".format(100*(distance_greater_than_50m/processed_df_wayne.shape[0])))


In [ ]:
fig, ax = plt.subplots(figsize=(17,9))

ax.set_title("Histogram of APT to centroid distance on/not on BFP")
ax.set_xlabel("APT point to centroid distance(meters)")
ax.set_ylabel("counts")

frqTrue, edgesTrue = np.histogram(processed_df_wayne['APT_to_Centroid_distance'].loc[processed_df_wayne['APT_to_Centroid_distance']<150].values, bins = np.arange(0,150,10))
p1 = ax.bar(edgesTrue[:-1], frqTrue, width=np.diff(edgesTrue), edgecolor="black", align="edge",alpha=0.4,label='Address Points on Rooftop',color='orange')

plt.legend()
plt.show()
gc.collect()

### Merge both data 

In [ ]:
from glob import glob 
import os
import os.path as osp 
pkl_path = osp.join(data_path,state,city,'Apt_realignment')
pkl_files = glob(pkl_path+'/*/*.pkl')
print(pkl_files)

In [ ]:

## Show single bfp within Parcel data
df_one_bfp = pd.read_pickle(pkl_files[0])
df_one_bfp = df_one_bfp.reset_index(drop=True)
print("simple scenario sahpe",df_one_bfp.shape[0])

df_two_bfp = pd.read_pickle(pkl_files[1])
df_two_bfp = df_two_bfp.reset_index(drop=True)
print(" complx scenario shape:",df_two_bfp.shape[0])

merge_df= pd.concat([df_one_bfp,df_two_bfp])
print("Merged Shape",merge_df.shape)
merge_df.to_pickle(os.path.join(pkl_path,'FinalUpdated_APT_{}_{}.pkl'.format(state,city)))

## Dallas Texas 

In [ ]:
import os
import shapely
import numpy as np
import sys 
import pycoredb 
import pandas as pd 
import gc
from shapely import wkt
from matplotlib import pyplot as plt
from shapely.geometry import Point,mapping,Polygon,box,MultiPoint
import geopandas as gpd 
import seaborn as sns 
 
# for parallelization
sys.path.append('/mnt/c/Users/tandon/OneDrive - TomTom/Desktop/tomtom/Workspace/01_Rooftop_accuracy/APT-Realignment/src/apt_realignment')
from process_geometries import ProcessGeometricData  as ProcessAPT_data
from utils.haversine_distance import get_distance

data_path  = "/mnt/c/Users/tandon/OneDrive - TomTom/Desktop/tomtom/Workspace/01_Rooftop_accuracy/BFP_Analysis_USA/data/data"
state = "Texas"
city = "Dallas"
apt_data_path = os.path.join(data_path,state,"APT_2022_09_000_nam_usa_utx.shp" )
parcel_path = os.path.join(data_path,state,city,"Parcels_48113/Parcels_48113.shp")
building_geojson = os.path.join(data_path,state,'Texas.geojson')

### Single BFP within Parcel 

In [ ]:
output_path = os.path.join(data_path,state,city,'Apt_realignment') 
apt_preprocess = ProcessAPT_data(parcel_shapefile=parcel_path,
                building_shapefile=building_geojson,
                apt_shape_file=apt_data_path , 
                output_path=output_path)

processed_df_dallas = apt_preprocess.process_dataframe(bfp_count_per_parcel=1,filename='APT_realigned_usa_utx_dallas')
processed_df_dallas.head(10)

In [ ]:
distance_greater_than_50m = processed_df_dallas.loc[processed_df_dallas['APT_to_Centroid_distance']>50].shape[0]
print("Data points greater than 50m: {:.2f}%".format(100*(distance_greater_than_50m/processed_df_dallas.shape[0])))

distance_greater_than_50m = processed_df_dallas.loc[processed_df_dallas['APT_to_Centroid_distance']<50].shape[0]
print("Data points less than 50m: {:.2f}%".format(100*(distance_greater_than_50m/processed_df_dallas.shape[0])))


In [ ]:
fig, ax = plt.subplots(figsize=(17,9))

ax.set_title("Histogram of APT to centroid distance on/not on BFP")
ax.set_xlabel("APT point to centroid distance(meters)")
ax.set_ylabel("counts")

frqTrue, edgesTrue = np.histogram(processed_df_dallas['APT_to_Centroid_distance'].loc[processed_df_dallas['APT_to_Centroid_distance']<50].values, bins = np.arange(2,50,5))
p1 = ax.bar(edgesTrue[:-1], frqTrue, width=np.diff(edgesTrue), edgecolor="black", align="edge",alpha=0.4,label='Address Points on Rooftop',color='orange')

plt.legend()
plt.show()
gc.collect()

### Two BFP within Parcel 

In [ ]:
output_path = os.path.join(data_path,state,city,'Apt_realignment') 
apt_preprocess = ProcessAPT_data(parcel_shapefile=parcel_path,building_shapefile=building_geojson,
                                    apt_shape_file=apt_data_path , output_path=output_path)

processed_df_dallas = apt_preprocess.process_dataframe(bfp_count_per_parcel=2,filename='APT_realigned_usa_utx_dallas')
processed_df_dallas.head(10)

In [ ]:
distance_greater_than_50m = processed_df_dallas.loc[processed_df_dallas['APT_to_Centroid_distance']>50].shape[0]
print("Data points greater than 50m: {:.2f}%".format(100*(distance_greater_than_50m/processed_df_dallas.shape[0])))

distance_greater_than_50m = processed_df_dallas.loc[processed_df_dallas['APT_to_Centroid_distance']<50].shape[0]
print("Data points less than 50m: {:.2f}%".format(100*(distance_greater_than_50m/processed_df_dallas.shape[0])))


In [ ]:
fig, ax = plt.subplots(figsize=(17,9))

ax.set_title("Histogram of APT to centroid distance on/not on BFP")
ax.set_xlabel("APT point to centroid distance(meters)")
ax.set_ylabel("counts")

frqTrue, edgesTrue = np.histogram(processed_df_dallas['APT_to_Centroid_distance'].loc[processed_df_dallas['APT_to_Centroid_distance']<50].values, bins = np.arange(2,50,5))
p1 = ax.bar(edgesTrue[:-1], frqTrue, width=np.diff(edgesTrue), edgecolor="black", align="edge",alpha=0.4,label='Address Points on Rooftop',color='orange')

plt.legend()
plt.show()
gc.collect()

### Merge Data

In [ ]:
from glob import glob 
pkl_path = osp.join(data_path,state,city,'Apt_realignment')
pkl_files = glob(pkl_path+'/*/*.pkl')
print(pkl_files)

In [ ]:

## Show single bfp within Parcel data
df_one_bfp = pd.read_pickle(pkl_files[0])
df_one_bfp = df_one_bfp.reset_index(drop=True)
print("simple scenario sahpe",df_one_bfp.shape[0])

df_two_bfp = pd.read_pickle(pkl_files[1])
df_two_bfp = df_two_bfp.reset_index(drop=True)
print(" complx scenario shape:",df_two_bfp.shape[0])

merge_df= pd.concat([df_one_bfp,df_two_bfp])
print("Merged Shape",merge_df.shape)
merge_df.to_pickle(os.path.join(pkl_path,'FinalUpdated_APT_{}_{}.pkl'.format(state,city)))

### Compare MNR data with MSFT dBFP ata

In [ ]:
import os
import shapely
import numpy as np
import sys 

import pycoredb 
import pandas as pd 
import gc

from shapely import wkt
from matplotlib import pyplot as plt
from shapely.geometry import Point,mapping,Polygon,box,MultiPoint
import geopandas as gpd 
import seaborn as sns 

# for parallelization
sys.path.append('/mnt/c/Users/tandon/OneDrive - TomTom/Desktop/tomtom/Workspace/01_Rooftop_accuracy/APT-Realignment/src/apt_realignment')
from process_geometries import ProcessGeometricData  as ProcessAPT_data
from utils.haversine_distance import get_distance

In [ ]:
data_path  = "/mnt/c/Users/tandon/OneDrive - TomTom/Desktop/tomtom/Workspace/01_Rooftop_accuracy/BFP_Analysis_USA/data/data"
state = "Texas"
building_geojson = os.path.join(data_path,state,'Texas.geojson')
mnr_building_geojson = os.path.join(data_path,state,'BFP__2022_09_012_nam_usa_utx.shp')

In [ ]:
bfp_df = gpd.read_file(building_geojson)
bfp_df = gpd.GeoDataFrame(bfp_df,crs="EPSG:4326",geometry='geometry')
bfp_df = bfp_df.to_crs("epsg:4326")
print("Total number of data points collected from MNR database ",bfp_df.shape[0])
gc.collect()
bfp_df.head()

In [ ]:
mnr_bfp_df = gpd.read_file(mnr_building_geojson)
mnr_bfp_df = gpd.GeoDataFrame(mnr_bfp_df,crs="EPSG:4326",geometry='geometry')
mnr_bfp_df = mnr_bfp_df.to_crs("epsg:4326")
print("Total number of data points collected from MNR database ",mnr_bfp_df.shape[0])
gc.collect()
mnr_bfp_df.head()

In [ ]:
def get_buildingfootprint(val):
    return bfp_df['geometry'].loc[val]

join_df = gpd.sjoin(mnr_bfp_df, bfp_df, op='intersects', how='left')
join_df.dropna(inplace=True)
join_df['MSFT_building_geom'] = join_df['index_right'].apply(lambda x: get_buildingfootprint(x))


In [ ]:
def get_buildingfootprint(val):
    return bfp_df['geometry'].loc[val]

join_df.dropna(inplace=True)
join_df['MSFT_building_geom'] = join_df['index_right'].apply(lambda x: get_buildingfootprint(x))


In [ ]:
join_df

In [ ]:
def get_poly_iou(data: gpd.GeoSeries):
    try:
        intersect = data['geometry'].intersection(data['MSFT_building_geom']).area
        union = data['geometry'].union(data['MSFT_building_geom']).area
        iou = intersect / union
    except:
        iou = None
    return iou
join_df['IOU'] = join_df.apply(lambda x: get_poly_iou(x),axis=1)

In [ ]:
join_df.dropna(inplace=True)
print(join_df['IOU'].mean())


In [ ]:
join_df